In [2]:
import findspark
findspark.init("/home/sturupu1/spark")
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from operator import add
import math

#Creating Spark Context
sc = SparkContext.getOrCreate()
#Creating Spark Session
spark = SparkSession(sc)
#Readind data set(csv file),dropping null values and selecting only tabs and views column
tags_views = spark.read.csv("./Data/USvideos.csv", inferSchema = True, header = True).dropna().select("tags","views")
#Mapping by splitting tags with "|" character,
def tags_split(x):
    tags=x["tags"].split("|")
    result=[]
    for every in tags:
        #Associating view count as counter and applying log on view count because views count will be really large
        if not x["views"] or every==None:
            continue        
        result.append((every.strip("\"").lower(),math.log(int(x["views"]))))#Stripping unnecessary characters
    return tuple(result)
rdd1=tags_views.rdd.flatMap(tags_split).reduceByKey(add)#reduce by similar tags and adding its view count
#Top Tags are queried from RDD by Sorting RDD's in descending order of view count
toptags=rdd1.takeOrdered(35, key = lambda x: -x[1])
df=spark.createDataFrame(toptags)
#Writing back to Disk
df.repartition(1).write.csv(path="/home/sturupu1/spark/workspace/Project_Youtube_Sentiment_Analysis/output/trending.csv")
#Spark session
spark.stop()

In [2]:
%%HTML
<script type='text/javascript' src='https://prod-apnortheast-a.online.tableau.com/javascripts/api/viz_v1.js'></script><div class='tableauPlaceholder' style='width: 1324px; height: 716px;'><object class='tableauViz' width='1324' height='716' style='display:none;'><param name='host_url' value='https%3A%2F%2Fprod-apnortheast-a.online.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='&#47;t&#47;sameeraturupu' /><param name='name' value='Top30tags&#47;Sheet1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='showAppBanner' value='false' /></object></div>